# **Market Cycle**

A fourier aproximation is build to understands the cyclical behaviur through the intraannual price movement of an asset between medium to short periods.

- 1) Data importing and date time
- 2) Number of armonics selection and fourier fiting
- 3) Prediction Making and Conclusions

In [ ]:
!pip install yfinance

In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
import plotly.graph_objs as go
import plotly.offline as pyo

**1) Data import and date time**


In [ ]:
# Download gold price data from Yahoo Finance
ticket="GC=F"
stardate="2022-01-01"
enddate="2022-12-31"

gold_data = yf.download(ticket, start=stardate, end=enddate, interval="1d")

# Get adjusted close prices
adj_close = gold_data["Adj Close"]
adj_close.index = pd.date_range(start=stardate, periods=len(adj_close), freq="D")

# Create the trace for the gold price
gold_trace = go.Scatter(x=adj_close.index, y=adj_close.values, name=ticket)

# Create the layout
layout = go.Layout(title="Gold Price",
                   xaxis=dict(title="Date"),
                   yaxis=dict(title="Price ($)"))

# Create the figure
fig = go.Figure(data=[gold_trace], layout=layout)

# Show the figure
fig.show()


[*********************100%***********************]  1 of 1 completed


In [ ]:
# Download gold price data from Yahoo Finance
ticket = "GC=F"
startdate = "2022-01-01"
enddate = "2022-12-31"

gold_data = yf.download(ticket, start=startdate, end=enddate, interval="1d")

# Get adjusted close prices
adj_close = gold_data["Adj Close"]
adj_close.index = pd.date_range(start=startdate, end=enddate, freq="D")

# Create the trace for the gold price
gold_trace = go.Scatter(x=adj_close.index, y=adj_close.values, name=ticket)

# Create the layout
layout = go.Layout(title="Gold Price",
                   xaxis=dict(title="Date"),
                   yaxis=dict(title="Price ($)"))

# Create the figure
fig = go.Figure(data=[gold_trace], layout=layout)

# Show the figure
fig.show()


[*********************100%***********************]  1 of 1 completed


ValueError: ignored

**2) Number of armonics selection and fourier fiting**

In [ ]:
# Split into train and test sets
train_size = int(len(adj_close) * 0.8)
train_adj_close = adj_close[:train_size]
test_adj_close = adj_close[train_size:]

def fourier_series(x, y, n):
    T = (x[-1] - x[0]).days
    a0 = y.mean()
    an = np.zeros(n)
    bn = np.zeros(n)
    freqs = np.zeros(n)
    phases = np.zeros(n)
    for i in range(1, n+1):
        time_diff = (x - x[0]).total_seconds() / (24 * 60 * 60)  # convert to days
        w = 2 * np.pi * i / T
        an[i-1] = (2 / T) * np.trapz(y * np.cos(w * (time_diff + phase)), x=time_diff)
        bn[i-1] = (2 / T) * np.trapz(y * np.sin(w * time_diff), x=time_diff)
        freqs[i-1] = i / T
        phases[i-1] = np.arctan2(bn[i-1], an[i-1])
    return a0, an, bn, freqs, phases

# amount of harmonics (use 2)
n = 1
a0, an, bn, freqs, phases = fourier_series(train_adj_close.index, train_adj_close.values, n)

#phases = np.zeros_like(phases)

# Construct Fourier series for train sets
T_train = (train_adj_close.index[-1] - train_adj_close.index[0]).total_seconds() / (24 * 60 * 60)
w_train = 2 * np.pi / T_train
f_train = np.zeros(len(train_adj_close))
for i in range(1, n+1):
    phase = phases[i-1]
    f_train += an[i-1] * np.cos(i * w_train * ((train_adj_close.index - train_adj_close.index[0]).total_seconds() / (24 * 60 * 60) + phase)) 
    f_train += bn[i-1] * np.sin(i * w_train * (train_adj_close.index - train_adj_close.index[0]).total_seconds() / (24 * 60 * 60))
f_train += a0

# Construct Fourier series for test set
w_test = 2 * np.pi / T_train
f_test = np.zeros(len(test_adj_close))
for i in range(1, n+1):
    phase = phases[i-1]
    f_test += an[i-1] * np.cos(i * w_test * ((test_adj_close.index - test_adj_close.index[0]).total_seconds() / (24 * 60 * 60) + phase))
    f_test += bn[i-1] * np.sin(i * w_test * (test_adj_close.index - test_adj_close.index[0]).total_seconds() / (24 * 60 * 60))
f_test += a0

##################################################################################################
# Print R-squared values for train and test sets
print("R-squared for train set:", r2_score(train_adj_close, f_train))
print("R-squared for test set:", r2_score(test_adj_close, f_test))

import plotly.graph_objs as go
from plotly.subplots import make_subplots

# Create subplots with 1 row and 2 columns, with the second column width set to 40%
fig = make_subplots(rows=1, cols=2, column_widths=[0.8, 0.2], subplot_titles=("Gold Price Prediction", "Wavelength Spectrum"))

# Add the train and test set predictions to the first subplot
fig.add_trace(go.Scatter(x=adj_close.index, y=adj_close.values, name=ticket), row=1, col=1)
fig.add_trace(go.Scatter(x=train_adj_close.index, y=f_train, name="Train Prediction"), row=1, col=1)
fig.add_trace(go.Scatter(x=test_adj_close.index, y=f_test, name="Test Prediction"), row=1, col=1)

# Add the wavelength spectrum to the second subplot
fig.add_trace(go.Scatter(x=1/freqs, y=np.sqrt(an**2 + bn**2), mode='markers', name='Wavelength Spectrum'), row=1, col=2)
for i in range(1, n+1):
    fig.add_shape(type="line", x0=T_train/i, y0=0, x1=T_train/i, y1=np.sqrt(an[i-1]**2 + bn[i-1]**2), line=dict(color='red', dash='dash'), row=1, col=2)

# Set the subplot titles and axis labels
fig.update_layout(title="Gold Price Prediction with Fourier Series",
                  xaxis_title="Date",
                  yaxis_title="Price ($)",
                  title_font=dict(size=24),
                  font=dict(size=16),
                  legend=dict(font=dict(size=14)))

fig.update_xaxes(title_text="Wavelength (days)", row=1, col=2)
fig.update_yaxes(title_text="Amplitude", row=1, col=2)

# Show the figure
fig.show()

R-squared for train set: 0.7758904726644311
R-squared for test set: -1.8860172768907963


In [ ]:
#measure mean error(error)
#consider growth(trend)(not needed on short term(no growth no inflation(dollars), good for assets that grow/fall little))

**3) Prediction Making and Conclusions**

In [ ]:
# Define start and end date for prediction
pred_start_date = enddate
pred_end_date = "2023-03-31"

# Define new date range for prediction
pred_index = pd.date_range(start=pred_start_date, end=pred_end_date, freq="D")

# Extend Fourier series to include predictions

w_pred = 2 * np.pi / T_train
f_pred = np.zeros(len(pred_index))
for i in range(1, n+1):
    phase = phases[i-1]
    f_pred += an[i-1] * np.cos(i * w_pred * ((pred_index - pred_index[0]).total_seconds() / (24 * 60 * 60) + phase))
    f_pred += bn[i-1] * np.sin(i * w_pred * (pred_index - pred_index[0]).total_seconds() / (24 * 60 * 60))
f_pred += a0

# Concatenate train and test predictions with new predictions
full_pred = np.concatenate((f_train, f_test, f_pred))

import plotly.graph_objects as go

fig = go.Figure()

# Add gold price data
fig.add_trace(go.Scatter(
    x=adj_close.index,
    y=adj_close.values,
    name=ticket
))

# Add train prediction data
fig.add_trace(go.Scatter(
    x=train_adj_close.index,
    y=f_train,
    name="Train Prediction"
))

# Add test prediction data
fig.add_trace(go.Scatter(
    x=test_adj_close.index,
    y=f_test,
    name="Test Prediction"
))

# Add new prediction data
fig.add_trace(go.Scatter(
    x=pred_index,
    y=f_pred,
    name="Prediction"
))

# Set axis labels and title
fig.update_layout(
    xaxis_title="Date",
    yaxis_title="Price ($)",
    title="Gold Price Prediction with Fourier Series"
)

fig.show()


##**Conclusions**
- The Algorithm using fourier fitting of the data with 2 harmonics was able to produce a good fitting and future prediction from the test-data 
- The algorithm can be used for many other tickets, stocks, crypto, commodities, etc
- This method is best suitable for medium to short perod of time and also external macroeconomical factors or thers should be take accounted (ex:sudden interest rate change).

**Anexo: Loop for obtaining optimum amount of harmonics**

In [ ]:
# Define range of harmonics to test
n_values = np.arange(1, 11)

# Initialize lists to store R2 values and predicted values for each value of n
r2_values = []
f_test_values = []

# Loop over values of n
for n in n_values:
    # Compute Fourier series coefficients
    a0, an, bn, freqs, phases = fourier_series(train_adj_close.index, train_adj_close.values, n)

    # Construct Fourier series for train set
    T_train = (train_adj_close.index[-1] - train_adj_close.index[0]).total_seconds() / (24 * 60 * 60)
    w_train = 2 * np.pi / T_train
    f_train = np.zeros(len(train_adj_close))
    for i in range(1, n+1):
        phase = phases[i-1]
        f_train += an[i-1] * np.cos(i * w_train * (train_adj_close.index - train_adj_close.index[0]).total_seconds() / (24 * 60 * 60) + phase) + bn[i-1] * np.sin(i * w_train * (train_adj_close.index - train_adj_close.index[0]).total_seconds() / (24 * 60 * 60) + phase)
    f_train += a0

    # Construct Fourier series for test set
    w_test = 2 * np.pi / T_train
    f_test = np.zeros(len(test_adj_close))
    for i in range(1, n+1):
        phase = phases[i-1]
        f_test += an[i-1] * np.cos(i * w_test * (test_adj_close.index - test_adj_close.index[0]).total_seconds() / (24 * 60 * 60) + phase) + bn[i-1] * np.sin(i * w_test * (test_adj_close.index - test_adj_close.index[0]).total_seconds() / (24 * 60 * 60) + phase)
    f_test += a0

    # Compute R2 for test set
    r2 = r2_score(test_adj_close, f_test)
    
    # Store results for this value of n
    r2_values.append(r2)
    f_test_values.append(f_test)

# Find index of maximum R2 value
best_n_idx = np.argmax(r2_values)

# Extract best values
best_n = n_values[best_n_idx]
best_r2 = r2_values[best_n_idx]
best_f_test = f_test_values[best_n_idx]

print(best_n)
